In [ ]:
import glob
import signal
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import skimage
from cytoolz import compose

# import cv2
from matplotlib.offsetbox import AnnotationBbox, DrawingArea, OffsetImage, TextArea
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_doh, blob_log

from paulssonlab.projects.molecule_counting.matriarch_stub import permute_labels
from paulssonlab.projects.molecule_counting.segmentation import invert, segment

mpl.rcParams["figure.figsize"] = (10, 10)

In [ ]:
segment_phase = compose(segment, invert)

In [ ]:
filenames = glob.glob("/n/scratch2/jqs1/200228/*.nd2") + glob.glob(
    "/n/scratch2/jqs1/200305/*.nd2"
)
for i in range(len(filenames)):
    print(i, filenames[i])

In [ ]:
file = filenames[10]
print("file", file)
nd2 = nd2reader.ND2Reader(file)

In [ ]:
def get_stdevs_alpha(img_file, fov_indices, savetxt=False, filename=None):
    nd2 = nd2reader.ND2Reader(img_file)
    stdevs_alpha = []
    for fov_index in fov_indices:
        start = time.time()
        print(str(img_file), "fov index", fov_index)
        phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
        phasetime = time.time()
        print("time to get phase:", phasetime - start)
        fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
        fluortime = time.time()
        print("time to get fluor:", fluortime - phasetime)
        # Segment the image.
        seg = segment_phase(phase_img)
        segtime = time.time()
        print("time to seg:", segtime - phasetime)
        # Normalise the fluorescence image to make it easier to compare between images.
        # Min background fluorescence = 0, max in-cell fluorescence = 1
        # ***Does this make sense??***
        # 50th smallest and largest fluor values, in case of random outliers?
        min_fluor = np.amin(fluor_img)
        cells_fluor = np.ma.masked_where(seg == 0, fluor_img)
        cells_fluor_max = np.amax(cells_fluor)
        fluor_img_norm = (fluor_img - min_fluor) / (cells_fluor_max - min_fluor)
        #         print('fluor_img_norm min and max',np.amin(fluor_img_norm),np.amax(fluor_img_norm), np.shape(fluor_img))
        fluor_img_norm_1000 = 1000 * fluor_img_norm
        #         print('fluor_img_norm1000 min and max',np.amin(fluor_img_norm_1000),
        #               np.amax(fluor_img_norm_1000), np.shape(fluor_img_norm_1000))
        # Get number of cells in the image.
        normtime = time.time()
        print("norm time:", normtime - segtime)
        cells = np.amax(seg)
        # Get the standard deviation in fluorescence of each cell in the image.
        for i in range(cells):
            cell = np.ma.masked_where(seg != i + 1, fluor_img_norm_1000)
            stdevs_alpha.append(np.std(cell))
        if savetxt:
            np.savetxt(filename, stdevs_alpha)
        end = time.time()
        print("total time:", end - start)
    return stdevs_alpha

In [ ]:
class TimeoutException(Exception):  # Custom exception class
    pass


def timeout_handler(signum, frame):  # Custom signal handler
    raise TimeoutException


# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)


def get_stdevs_alpha_time_imgnorm(img_file, fov_indices, savetxt=False, filename=None):
    # Normalises each image. Rescale so background = 0 and max cell fluor = 1.
    nd2 = nd2reader.ND2Reader(img_file)
    stdevs_alpha = []
    fluor_diffs = []
    for fov_index in fov_indices:
        signal.alarm(80)
        try:
            print(str(img_file), "fov index", fov_index)
            phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
            fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
            # Segment the image.
            seg = segment_phase(phase_img)
            # Normalise the fluorescence image to make it easier to compare between images.
            # Min background fluorescence = 0, max in-cell fluorescence = 1
            # ***Does this make sense??***
            # 50th smallest and largest fluor values, in case of random outliers?
            min_fluor = np.amin(fluor_img)
            cells_fluor = np.ma.masked_where(seg == 0, fluor_img)
            cells_fluor_max = np.amax(cells_fluor)
            fluor_img_norm = (fluor_img - min_fluor) / (cells_fluor_max - min_fluor)
            print(np.amin(fluor_img_norm), np.amax(fluor_img_norm))
            #             fluor_img_norm_1000 = 1000*fluor_img_norm
            # Get number of cells in the image.
            cells = np.amax(seg)
            #             print('cells in field:',cells)
            # Get the standard deviation in fluorescence of each cell in the image.
            for i in range(cells):
                cell = np.ma.masked_where(seg != i + 1, fluor_img_norm)
                stdevs_alpha.append(np.std(cell))
                fluor_diffs.append(np.amax(cell) - np.amin(cell))
            #                 print(np.amin(cell),np.amax(cell), np.amax(cell)-np.amin(cell),np.std(cell))
            if savetxt:
                np.savetxt(filename + "_alpha.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs.csv", fluor_diffs)
        except TimeoutException:
            print("timeout")
            continue  # continue the for loop if function A takes more than 5 second
        else:
            # Reset the alarm
            signal.alarm(0)
    return stdevs_alpha


def get_stdevs_alpha_time_imgnorm2(img_file, fov_indices, savetxt=False, filename=None):
    # Normalises each image. Subtract background (non-cell) and
    # then divide by the mean fluorescence in the cells.
    nd2 = nd2reader.ND2Reader(img_file)
    stdevs_alpha = []
    fluor_diffs = []
    for fov_index in fov_indices:
        signal.alarm(80)
        try:
            print(str(img_file), "fov index", fov_index)
            phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
            fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
            # Segment the image.
            seg = segment_phase(phase_img)
            min_fluor = np.amin(fluor_img)
            cells_fluor = np.ma.masked_where(seg == 0, fluor_img)
            cells_fluor_mean = np.mean(cells_fluor)
            fluor_img_norm = (fluor_img - min_fluor) / cells_fluor_mean
            print(np.amin(fluor_img_norm), np.amax(fluor_img_norm))
            # Get number of cells in the image.
            cells = np.amax(seg)
            # Get the standard deviation in fluorescence of each cell in the image.
            for i in range(cells):
                cell = np.ma.masked_where(seg != i + 1, fluor_img_norm)
                stdevs_alpha.append(np.std(cell))
                fluor_diffs.append(np.amax(cell) - np.amin(cell))
            #                 print(np.amin(cell),np.amax(cell), np.amax(cell)-np.amin(cell),np.std(cell))
            if savetxt:
                np.savetxt(filename + "_alpha2.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs2.csv", fluor_diffs)
        except TimeoutException:
            if savetxt:
                np.savetxt(filename + "_alpha2.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs2.csv", fluor_diffs)
            print("timeout")
            continue  # continue the for loop if function A takes more than 5 second
        else:
            # Reset the alarm
            signal.alarm(0)
    return stdevs_alpha


def get_stdevs_alpha_time_cellnorm(
    img_file, fov_indices, time_allowed=80, savetxt=False, filename=None
):
    # Normalises each image. Subtract background (non-cell) and
    # then divide by the mean fluorescence in the specific cell.
    nd2 = nd2reader.ND2Reader(img_file)
    stdevs_alpha = []
    fluor_diffs = []
    for fov_index in fov_indices:
        signal.alarm(time_allowed)
        try:
            print(str(img_file), "fov index", fov_index)
            phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
            fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
            # Segment the image.
            seg = segment_phase(phase_img)
            min_fluor = np.amin(fluor_img)
            # Get number of cells in the image.
            cells = np.amax(seg)
            # Get the standard deviation in fluorescence of each cell in the image.
            for i in range(cells):
                cell = np.ma.masked_where(seg != i + 1, fluor_img)
                cell = (cell - min_fluor) / np.mean(cell)
                stdevs_alpha.append(np.std(cell))
                fluor_diffs.append(np.amax(cell) - np.amin(cell))
            #                 print(np.amin(cell),np.amax(cell), np.amax(cell)-np.amin(cell),np.std(cell))
            if savetxt:
                np.savetxt(filename + "_alpha_cellnorm.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs_cellnorm.csv", fluor_diffs)
        except TimeoutException:
            if savetxt:
                np.savetxt(filename + "_alpha_cellnorm.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs_cellnorm.csv", fluor_diffs)
            print("timeout")
            continue  # continue the for loop if function A takes more than 5 second
        else:
            # Reset the alarm
            signal.alarm(0)
    return stdevs_alpha

In [ ]:
files_indices = [2, 9, 10, 15, 39, 44, 45, 5, 11, 16, 12, 0, 4, 3, 8, 18, 38, 17, 55]
files_names = [
    "chr_P_sfGFP",
    "chr_P_GFPmut3",
    "chr_P_Venus",
    "chr_P_Dendra2",
    "plas_X_DHL_GFPmut3",
    "plas_P_DHL_sfGFP40to54",
    "plas_X_arai_sfGFP",
    "chr_P_GFPmut3_40x",
    "chr_P_sfGFP_40x",
    "chr_P_Venus_40x",
    "chr_P_Dendra2_40x",
    "chr_P_sfGFP_20x",
    "chr_P_GFPmut3_20x",
    "chr_P_Dendra2_20x",
    "chr_P_Venus_20x",
    "plas_X_DHL_GFPmut3_20x",
    "plas_X_DHL_GFPmut3_40x",
    "plas_P_DHL_sfGFP_20x",
    "plas_P_DHL_sfGFP_40x",
]

# for i in range(18,len(files_indices)):
#     get_stdevs_alpha_time_cellnorm(filenames[files_indices[i]],np.arange(5),time_allowed=200,
#                                    savetxt=True,filename = files_names[i])

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(30, 15), sharex=True)
ax = axes.ravel()
logmeans = []
logstds = []
for i in range(len(files_names)):
    try:
        data_stdevs = np.loadtxt(
            "202004_cellnorm/" + files_names[i] + "_alpha_cellnorm.csv"
        )
        data_fluordiffs = np.loadtxt(
            "202004_cellnorm/" + files_names[i] + "_fluordiffs_cellnorm.csv"
        )
        ax[i].hist(data_stdevs, bins=np.logspace(-1.5, 0.5, 50))
        #         ax[i].hist(data_fluordiffs, bins = 50)
        ax[i].set_title(files_names[i])
        plt.xscale("log")
        print(files_names[i], "number of cells:", len(data_stdevs))
        print(files_names[i], "mean of stdevs:", np.mean(data_stdevs))
        print(files_names[i], "stdev of stdevs:", np.std(data_stdevs))
        print(
            files_names[i],
            "mean of log10 stdevs:",
            np.mean(np.log10(data_stdevs)),
            10 ** (np.mean(np.log10(data_stdevs))),
        )
        logmeans.append(10 ** (np.mean(np.log10(data_stdevs))))
        print(
            files_names[i],
            "stdev of log10 stdevs:",
            np.std(np.log10(data_stdevs)),
            10 ** (np.std(np.log10(data_stdevs))),
        )
        logstds.append(10 ** (np.std(np.log10(data_stdevs))))
    except OSError:
        pass

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
ax = axes.ravel()
for i in range(len(logstds)):
    if logstds[i] < 1.7:
        ax[0].plot(logmeans[i], logstds[i], "bo")
    else:
        ax[0].plot(logmeans[i], logstds[i], "ro")
ax[0].axhline(1.7, linestyle="--", color="k")
for i in range(len(logstds)):
    if logstds[i] < 1.7:
        ax[1].plot(logmeans[i], logstds[i], "bo")
    else:
        ax[1].plot(logmeans[i], logstds[i], "ro")
ax[1].axhline(1.7, linestyle="--", color="k")
for i, txt in enumerate(files_names):
    ax[1].annotate(txt, (logmeans[i] + 0.001, logstds[i] + 0.05))
plt.xlabel("Mean of cell stdevs")
plt.ylabel("Stdev of cell stdevs")

In [ ]:
for i in range(len(files_names)):
    if logstds[i] < 1.7:
        print(files_names[i])

In [ ]:
files_names = [
    "chr_P_sfGFP",
    "chr_P_GFPmut3",
    "chr_P_Venus",
    "chr_P_Dendra2",
    "plas_X_DHL_GFPmut3",
    "plas_P_DHL_sfGFP40to54",
    "plas_X_arai_sfGFP",
    "chr_P_GFPmut3_40x",
    "chr_P_sfGFP_40x",
    "chr_P_Venus_40x",
    "chr_P_Dendra2_40x",
    "chr_P_sfGFP_20x",
    "chr_P_GFPmut3_20x",
    "chr_P_Dendra2_20x",
    "chr_P_Venus_20x",
    "plas_X_DHL_GFPmut3_20x",
    "plas_X_DHL_GFPmut3_40x",
    "plas_P_DHL_sfGFP_20x",
    "plas_P_DHL_sfGFP_40x",
]

fig, axes = plt.subplots(3, 1, figsize=(3, 9), sharex=True)
ax = axes.ravel()
# names = ['chr_P_sfGFP', 'chr_P_sfGFP_40x', 'chr_P_sfGFP_20x']
# names = ['chr_P_GFPmut3', 'chr_P_GFPmut3_40x', 'chr_P_GFPmut3_20x']
# names = ['chr_P_Venus', 'chr_P_Venus_40x', 'chr_P_Venus_20x']
# names = ['chr_P_Dendra2', 'chr_P_Dendra2_40x', 'chr_P_Dendra2_20x']
# names = ['plas_X_DHL_GFPmut3', 'plas_X_DHL_GFPmut3_40x', 'plas_X_DHL_GFPmut3_20x']
# names = ['plas_P_DHL_sfGFP40to54', 'plas_P_DHL_sfGFP_40x', 'plas_P_DHL_sfGFP_20x']

base = names[1][:-4]
ax[0].set_title(base + " 100x")
ax[1].set_title(base + " 40x")
ax[2].set_title(base + " 20x")

for i in range(len(names)):
    try:
        data_stdevs = np.loadtxt("202004_cellnorm/" + names[i] + "_alpha_cellnorm.csv")
        data_fluordiffs = np.loadtxt(
            "202004_cellnorm/" + names[i] + "_fluordiffs_cellnorm.csv"
        )
        ax[i].hist(data_stdevs, bins=np.logspace(-1.5, 0.5, 50))
        plt.xscale("log")
    except OSError:
        pass
fig.savefig(base)